In [1]:
import pandas as pd
from itertools import combinations
import itertools

In [2]:
df = pd.read_csv('data.txt')
deletingCol = []
for i in df.columns:
    if i not in ['CustServ Calls', "Int'l Plan", 'VMail Plan', 'Churn?']:
        deletingCol.append(i)
df = df.drop(deletingCol, axis=1)
df.columns
df

,Int'l Plan,VMail Plan,CustServ Calls,Churn?
0,no,yes,1,False.
1,no,yes,1,False.
2,no,no,0,False.
3,yes,no,2,False.
4,yes,no,3,False.
...,...,...,...,...
3328,no,yes,2,False.
3329,no,no,3,False.
3330,no,no,2,False.
3331,yes,no,2,False.


In [3]:
df = df.rename(columns={'CustServ Calls':'CustServCalls',"Int'l Plan":'IntlPlan', 'VMail Plan':'VMailPlan', 'Churn?':'Churn'})
df

,IntlPlan,VMailPlan,CustServCalls,Churn
0,no,yes,1,False.
1,no,yes,1,False.
2,no,no,0,False.
3,yes,no,2,False.
4,yes,no,3,False.
...,...,...,...,...
3328,no,yes,2,False.
3329,no,no,3,False.
3330,no,no,2,False.
3331,yes,no,2,False.


In [4]:
df = df.sort_values(by=['CustServCalls'], ascending=False)
df = df.replace({'IntlPlan': {'yes': True, 'no': False}})
df = df.replace({'VMailPlan': {'yes': True, 'no': False}})
df['Churn'] = df['Churn'].astype(str)
df = df.replace({'Churn': {'True.': True, 'False.': False}})
df

,IntlPlan,VMailPlan,CustServCalls,Churn
542,True,True,9,True
2380,False,False,9,True
1502,False,True,8,False
1912,False,False,8,True
902,False,False,7,False
...,...,...,...,...
2760,True,False,0,True
806,False,False,0,False
805,False,False,0,False
804,False,False,0,False


In [5]:
df = df.replace({'CustServCalls': [1,2,3,4,5,6,7,8,9]}, {'CustServCalls': True})
df = df.replace({'CustServCalls': [0]}, {'CustServCalls': False})
df

,IntlPlan,VMailPlan,CustServCalls,Churn
542,True,True,True,True
2380,False,False,True,True
1502,False,True,True,False
1912,False,False,True,True
902,False,False,True,False
...,...,...,...,...
2760,True,False,False,True
806,False,False,False,False
805,False,False,False,False
804,False,False,False,False


In [6]:
items = ['IntlPlan', 'VMailPlan', 'CustServCall', 'Churn']
itemsFrequency = dict.fromkeys(items,0)
baskets = []
frequentItems = {}
print(itemsFrequency)
for i in df.itertuples(): 
    basket= []
    if i[1] == True:
        itemsFrequency['IntlPlan'] += 1
        basket.append('IntlPlan')
    if i[2] == True:
        itemsFrequency['VMailPlan'] += 1
        basket.append('VMailPlan')
    if i[3] == True:
        itemsFrequency['CustServCall'] += 1
        basket.append('CustServCall')
    if i[4] == True:
        itemsFrequency['Churn'] += 1
        basket.append('Churn')
    baskets.append(basket)
print(itemsFrequency)
support = 0.01*df.shape[0]
print(f"support is : {support}")
L1 = []
for key,value in itemsFrequency.items():
    if value >= support:
        frequentItems[key] = value
        L1.append(key)
print(L1)

{'IntlPlan': 0, 'VMailPlan': 0, 'CustServCall': 0, 'Churn': 0}
{'IntlPlan': 323, 'VMailPlan': 922, 'CustServCall': 2636, 'Churn': 483}
support is : 33.33
['IntlPlan', 'VMailPlan', 'CustServCall', 'Churn']


In [7]:
c2 = [tuple(sorted(x)) for x in list(combinations(L1,2))]
pairFrequency = dict.fromkeys(c2,0)
L2 = []
frequentPairs = {}

for basket in baskets:
    basket_pairs = [tuple(sorted(x)) for x in list(combinations(basket,2))]
    for duo in basket_pairs:
        if duo in c2:
            pairFrequency[duo] += 1
for key,value in pairFrequency.items():
    if value >= support :
        frequentPairs[key]=value
        L2.append(key)
print(L2)
L2

[('IntlPlan', 'VMailPlan'), ('CustServCall', 'IntlPlan'), ('Churn', 'IntlPlan'), ('CustServCall', 'VMailPlan'), ('Churn', 'VMailPlan'), ('Churn', 'CustServCall')]


[('IntlPlan', 'VMailPlan'),
 ('CustServCall', 'IntlPlan'),
 ('Churn', 'IntlPlan'),
 ('CustServCall', 'VMailPlan'),
 ('Churn', 'VMailPlan'),
 ('Churn', 'CustServCall')]

In [8]:
c3 = []
L3 = []
frequentTriples = {}

for pair in list(combinations(L2,2)):
    triple = set(pair[0]).union(set(pair[1]))
    if len(triple)==3 and all(tuple(sorted(x)) in L2 for x in list(combinations(triple,2))):
        c3.append(tuple(sorted(triple)))
        
tripleFrequency = dict.fromkeys(c3,0)

for basket in baskets:
    subs = [tuple(sorted(x)) for x in list(combinations(basket,3))]
    for trio in subs:
        if trio in c3:
            tripleFrequency[trio] += 1
            
for key,value in tripleFrequency.items():
    if value >= support :
        frequentTriples[key] = value
        L3.append(key)
print(L3)

[('CustServCall', 'IntlPlan', 'VMailPlan'), ('Churn', 'IntlPlan', 'VMailPlan'), ('Churn', 'CustServCall', 'IntlPlan'), ('Churn', 'CustServCall', 'VMailPlan')]


In [9]:
c4 = []
L4 = []
frequentQuadro = {}

for pair in list(combinations(L3,2)):
    quadro = set(pair[0]).union(set(pair[1]))
    if len(quadro)==4 and all(tuple(sorted(x)) in L3 for x in list(combinations(quadro,2))):
        c4.append(tuple(sorted(quadro)))
        
quadroFrequency = dict.fromkeys(c4,0)

for basket in baskets:
    subs = [tuple(sorted(x)) for x in list(combinations(basket,4))]
    for quadro in subs:
        if quadro in c4:
            quadroFrequency[quadro] += 1
            
for key,value in quadroFrequency.items():
    if value >= support :
        frequentQuadro[key] = value
        L4.append(key)
print(c4)
print(L4)

[]
[]


In [10]:
def find_rule_1(confidence, support, key, value, key1, key2):
    output = []
    lhs = key[key1]
    rhs = key[key2]
    lhs_sup = float(frequentItems[lhs])/n
    rhs_sup = float(frequentItems[rhs])/n
    rule_sup = float(value)/n
    conf = rule_sup/lhs_sup
    lift = conf/rhs_sup
    if conf >= confidence and lhs_sup > support:
        output.append({'lhs':lhs,
                     'rhs':rhs,
                     'lhs_support': lhs_sup,
                     'rule_support': rule_sup,
                     'rule_confidence':conf,
                     'lift':lift})
    return output

In [11]:
support = 0.01
confidence = 0.05
n = len(baskets)

rules_1 = []

for key,value in frequentPairs.items():
    if 'Churn' in key:
        rules_1.append(find_rule_1(confidence, support, key, value, 1, 0))
    else:
        rules_1.append(find_rule_1(confidence, support, key, value, 0, 1))
        rules_1.append(find_rule_1(confidence, support, key, value, 1, 0))

print(len(rules_1))
for i in rules_1:
    print(i)

9
[{'lhs': 'IntlPlan', 'rhs': 'VMailPlan', 'lhs_support': 0.0969096909690969, 'rule_support': 0.027602760276027604, 'rule_confidence': 0.2848297213622291, 'lift': 1.0296501749461058}]
[{'lhs': 'VMailPlan', 'rhs': 'IntlPlan', 'lhs_support': 0.27662766276627665, 'rule_support': 0.027602760276027604, 'rule_confidence': 0.09978308026030368, 'lift': 1.0296501749461058}]
[{'lhs': 'CustServCall', 'rhs': 'IntlPlan', 'lhs_support': 0.7908790879087909, 'rule_support': 0.07200720072007201, 'rule_confidence': 0.09104704097116843, 'lift': 0.9395039862442861}]
[{'lhs': 'IntlPlan', 'rhs': 'CustServCall', 'lhs_support': 0.0969096909690969, 'rule_support': 0.07200720072007201, 'rule_confidence': 0.7430340557275542, 'lift': 0.9395039862442861}]
[{'lhs': 'IntlPlan', 'rhs': 'Churn', 'lhs_support': 0.0969096909690969, 'rule_support': 0.0411041104110411, 'rule_confidence': 0.4241486068111455, 'lift': 2.9268888333365384}]
[{'lhs': 'CustServCall', 'rhs': 'VMailPlan', 'lhs_support': 0.7908790879087909, 'rule_s

In [12]:
def find_rule_2(confidence, support, key, value, key1, key2, key3):
    output = []
    rhs = tuple(sorted([key[key1], key[key2]]))
    lhs = tuple(sorted([key[key3]]))
    lhs_sup = float(frequentItems[lhs[0]])/n
    rhs_sup = float(frequentPairs[tuple(sorted(x for x in rhs))])/n
    rule_sup = float(value)/n
    conf = rule_sup/lhs_sup
    lift = conf/rhs_sup
    if conf >= confidence and lhs_sup > support:
        output.append({'lhs':lhs,
                     'rhs':rhs,
                     'lhs_support': lhs_sup,
                     'rule_support': rule_sup,
                     'rule_confidence':conf,
                     'lift':lift})
    return output

In [13]:
support = 0.01
confidence = 0.05
n = len(baskets)

rules_2 = []

for key,value in frequentTriples.items():
    if 'Churn' in key:
        rules_2.append(find_rule_2(confidence, support, key, value,0,1,2))
        rules_2.append(find_rule_2(confidence, support, key, value, 0,2,1))
    else:
        rules_2.append(find_rule_2(confidence, support, key, value,0,1,2))
        rules_2.append(find_rule_2(confidence, support, key, value,0,2,1))
        rules_2.append(find_rule_2(confidence, support, key, value,1,2,0))

rules_2 = [x for x in rules_2 if x != []]        
print(len(rules_2))
for i in rules_2:
    print(i)

5
[{'lhs': ('VMailPlan',), 'rhs': ('CustServCall', 'IntlPlan'), 'lhs_support': 0.27662766276627665, 'rule_support': 0.019801980198019802, 'rule_confidence': 0.07158351409978307, 'lift': 0.9941160520607374}]
[{'lhs': ('IntlPlan',), 'rhs': ('CustServCall', 'VMailPlan'), 'lhs_support': 0.0969096909690969, 'rule_support': 0.019801980198019802, 'rule_confidence': 0.2043343653250774, 'lift': 0.9342200817949012}]
[{'lhs': ('IntlPlan',), 'rhs': ('Churn', 'VMailPlan'), 'lhs_support': 0.0969096909690969, 'rule_support': 0.010801080108010801, 'rule_confidence': 0.11145510835913314, 'lift': 4.643498452012385}]
[{'lhs': ('IntlPlan',), 'rhs': ('Churn', 'CustServCall'), 'lhs_support': 0.0969096909690969, 'rule_support': 0.029402940294029404, 'rule_confidence': 0.30340557275541796, 'lift': 2.5863191150736777}]
[{'lhs': ('VMailPlan',), 'rhs': ('Churn', 'CustServCall'), 'lhs_support': 0.27662766276627665, 'rule_support': 0.020102010201020103, 'rule_confidence': 0.07266811279826464, 'lift': 0.61944455231

In [14]:
rules_1 = [i[0] for i in rules_1]
rules_2 = [i[0] for i in rules_2]
results = rules_1+rules_2
output = pd.DataFrame(results)
output = output.sort_values('lift',ascending=False)
output

,lhs,rhs,lhs_support,rule_support,rule_confidence,lift
11,"(IntlPlan,)","(Churn, VMailPlan)",0.096910,0.010801,0.111455,4.643498
4,IntlPlan,Churn,0.096910,0.041104,0.424149,2.926889
12,"(IntlPlan,)","(Churn, CustServCall)",0.096910,0.029403,0.303406,2.586319
0,IntlPlan,VMailPlan,0.096910,0.027603,0.284830,1.029650
1,VMailPlan,IntlPlan,0.276628,0.027603,0.099783,1.029650
8,CustServCall,Churn,0.790879,0.117312,0.148331,1.023575
5,CustServCall,VMailPlan,0.790879,0.218722,0.276555,0.999739
6,VMailPlan,CustServCall,0.276628,0.218722,0.790672,0.999739
9,"(VMailPlan,)","(CustServCall, IntlPlan)",0.276628,0.019802,0.071584,0.994116
2,CustServCall,IntlPlan,0.790879,0.072007,0.091047,0.939504


In [16]:
tripleFrequency

{('CustServCall', 'IntlPlan', 'VMailPlan'): 66,
 ('Churn', 'IntlPlan', 'VMailPlan'): 36,
 ('Churn', 'CustServCall', 'IntlPlan'): 98,
 ('Churn', 'CustServCall', 'VMailPlan'): 67}